In [3]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from rnamodif.data_utils.dataloading import get_valid_dataset_unlimited
from rnamodif.data_utils.workers import worker_init_fn_multisplit, worker_init_simple_fn
from rnamodif.architectures.rodan_pretrained_modcaller import RodanPretrainedModcaller
from rnamodif.data_utils.split_methods import kfold_split_single
from rnamodif.data_utils.dataloading import get_test_dataset
from rnamodif.data_utils.dataloading2 import nanopore_datamodule
from rnamodif.data_utils.split_methods import get_kfold_splits, get_fullvalid_split, get_valid_portions
import torch



# 'm6A_0_covid':'neg', 
# 'UNM_novoa_1':'neg',
# 'UNM_novoa_2':'neg',
# 'UNM_novoa_short':'neg',    
# 'm6A_5_covid':'pos',
# 'm6A_33_covid':'pos',
# 'm6A_10_covid':'pos',
# 'm6a_novoa_1':'pos',
# 'm6a_novoa_2':'pos',

exp='m6A_33_covid'
label='pos'
num_label = 1
architecture=RodanPretrainedModcaller
checkpoint = '../checkpoints_pl/m6a_base_novoa_weighted/last.ckpt'

valid_files=kfold_split_single(total_k=5, current_k=0)(exp, label=label)[f'valid_{label}_files']
test_dset = get_test_dataset(valid_files, window=4096, normalization='rodan', trim_primer=False, stride=4096, read_limit=None)
test_loader = DataLoader(test_dset, batch_size=4, num_workers=1, pin_memory=True, worker_init_fn = worker_init_simple_fn) 

model = architecture().load_from_checkpoint(checkpoint)
trainer = pl.Trainer(accelerator='gpu', profiler=None, precision=16)
    
torch.set_grad_enabled(False)
model.cuda()
model.eval()
for batch_idx, batch in enumerate(test_loader):
    batch[0] = batch[0].cuda()
    x, idx = batch
    y = torch.tensor([num_label,num_label,num_label,num_label]).cuda()
    batch = (x,y,idx)
    pred, real, loss = model.training_step(batch, batch_idx)
    # loss = model.predict_step(batch, batch_idx, expanded=True)
    
    # print(pred.min())
    # print(pred.max())
    # print(pred)
    print(loss)
    break
    
    
#compute loss only from A positions - skip all others????

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


TEMPORARY
None


In [4]:
# pred, real
pred[:,0,:].argmax(dim=-1)
#TODO dont compute loss from Ns
# print(pred[:,0,:])

# pred

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 2, 0, 0, 0, 2,
        2, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0,
        0, 4, 0, 4, 4, 0, 0, 0, 4, 4, 0, 1, 1, 0, 0, 1, 1, 0, 0, 3, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 4, 0, 1, 0,
        0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        2, 2, 0, 0, 0, 2, 2, 4, 2, 2, 2, 0, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2,
        2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 4,

In [ ]:
# "vocab": [ '<PAD>', 'A', 'C', 'G', 'T' ]}